## FHNW bverI - HS2023

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Machine Learning Recap

## Lernziele

- Machine Learning Refresher zu Data-Preprocessing, Modell-Selektion und Evaluierung

## Setup

Im Folgenden installieren und laden wir die benötigten Python packages. Danach setzten wir die Pfade für den Zugriff auf Daten und spezifizieren einen Output-Folder.

In [ ]:
import os
from pathlib import Path

Mount your google drive to store data and results.

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

print(f"In colab: {IN_COLAB}")

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Modifizieren Sie die folgenden Pfade bei Bedarf.

In [ ]:
if IN_COLAB:
    DATA_PATH = Path('/content/drive/MyDrive/bverI/data')
else:
    DATA_PATH = Path('../data')

Install packages not in base Colab environment.

In [ ]:
if IN_COLAB:
    os.system("pip install torchshow")

## Machine Learning Recap

In diesem Teil geht es darum wieder mit _Machine Learning_ vertraut zu werden. Wir werden ein einfaches Modell trainieren und dessen Performance evaluieren. Es geht noch nicht darum die einzelnen _PyTorch_ Befehle zu kennen, dazu kommen wir später.

Im Wesentlichen besteht Machine Learning aus den folgenden Schritten.

- Daten: Wie sollen Features / Messwerte einen Datenpunkt beschreiben? Wie muss ein Datensatz prozessiert werden?
- Modell: Wie soll eine Fragestellung als Modell repräsentiert werden?
- Optimisierung: Wie finde ich für ein gegebenes Modell die optimalen Modellparameter?
- Evaluierung: Wie soll ein Modell gemessen werden? Wie soll zwischen verschiedenen Modellen das beste ausgewählt werden?
t)

In [ ]:
from collections import Counter

from matplotlib import pyplot as plt
import skimage
import torch
import torchshow as ts
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

### Ziel

Im Folgenden ist es unser Ziel ein Klassifikations-Modell zu trainieren mit dem man Bilder einer von 10 Klassen zuordnen kann.

### Daten

Um ein Modell zu trainieren brauchen wir einen Datensatz. Wir finden die 10 Klassen im `CIFAR10` Datensatz, der praktischerweise im `torchvision` Package enthalten ist.

Der Datensatz wird direkt in `DATA_PATH` abgelegt.

In [ ]:
# Transform data to tensor
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root=DATA_PATH, train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root=DATA_PATH, train=False, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Nun geht es darum die Daten zu visualisieren. Wir schauen uns die 16 ersten Bilder an.

In [ ]:
images, labels = next(iter(torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=False)))

def plot_square_collage_with_captions(images: list[torch.Tensor], captions: list[str], caption_width=30):
    """Plot Square collage with captions."""
    import math
    from textwrap import wrap
    num_images = len(images)
    side_length = math.ceil(math.sqrt(num_images))
    
    plt.figure(figsize=(8, 8))
    for i in range(num_images):
        ax = plt.subplot(side_length, side_length, i + 1)
        caption = captions[i]
        caption = "\n".join(wrap(caption, caption_width))
        plt.title(caption)
        plt.imshow(images[i])
        plt.axis("off")
     
images_list =  [x.squeeze(0).permute(1, 2, 0) for x in torch.split(images, 1)]

plot_square_collage_with_captions(images_list, captions=[classes[i] for i in labels])        

Schauen Sie sich die Labels an: Sind diese korrekt?

Mit dem folgenden Befehl sehen wir, dass die Bilder eine Auflösung von 32x32 Pixeln haben.

(16 Bilder, 3 Farb-Channels, 32 Höhe, 32 Breite)

In [ ]:
images.shape

Eine noch schnellere Möglichkeit Bilder anzuschauen gibt es mit dem Package `torchshow` (jedoch ohne die Labels).

In [ ]:
ts.show(images)

### Modell & Optimisierung

Nun geht es darum Modelle zu definieren, mit denen man lernen kann Bilder zu klassifizieren.

Als erstes implementieren wir die Modelle:

In [ ]:
class CentroidClassifier:

    def __init__(self, num_classes, shape):
        self.num_classes = num_classes
        self.shape = shape

    def fit(self, X, y):
        class_sum = torch.zeros((self.num_classes,) + self.shape)
        class_counts = torch.zeros(self.num_classes)
        for i in range(self.num_classes):
            class_sum[i] += torch.sum(X[y == i], dim=0)
            class_counts[i] += torch.sum(y == i)
        centroids = class_sum / class_counts.view(self.num_classes, 1, 1, 1)
        self.centroids = centroids
        return self
    
    def predict(self, X):
        centroids = self.centroids.view(self.num_classes, -1)
        X = X.view(X.size(0), -1)
        # Compute distances to centroids
        dists = torch.cdist(X, centroids)
        return torch.argmin(dists, dim=1)

**FRAGE**: Was macht dieses Modell? Interpretieren Sie die Klasse `CentroidModel`

**FRAGE**: Welche Modell-Parameter werden gelernt / optimisiert?

In [ ]:
class ClosestNeighborClassifier:

    def __init__(self):
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        return self
    
    def predict(self, X):
        # Reshape input images
        X_flat = X.view(X.shape[0], -1)
        X_train_flattened = self.X_train.view(self.X_train.shape[0], -1)
        
        # Compute distances to all training samples
        dists = torch.cdist(X_flat, X_train_flattened)
        
        # Get the indices of the minimum distances
        indices_of_min_dists = torch.argmin(dists, dim=1)
        
        # Return the labels of the closest training samples
        return self.y_train[indices_of_min_dists]


**FRAGE**: Was macht dieses Modell? Interpretieren Sie die Klasse `ClosestNeighborClassifier` 

**FRAGE**: Welche Modell-Parameter werden gelernt / optimisiert?

In [ ]:
from skimage.feature import hog

class HOGClassifier:

    def __init__(self, k):
        # Initialize the class with a given number k for k-nearest neighbors
        self.k = k

    def fit(self, X, y):
        features = []
        # Loop over each image in X to compute its HOG feature
        for i in tqdm(range(0, X.shape[0])):
            fd = self._hog(X[i])
            features.append(torch.tensor(fd))
            
        self.features_train = torch.stack(features)
        self.y_train = y
        return self

    def predict(self, X):
        features = []
        # Loop over each image in X to compute its HOG feature
        for i in range(0, X.shape[0]):
            fd = self._hog(X[i]) 
            features.append(torch.tensor(fd))

        features_test = torch.stack(features)
        
        # Compute L2 distances between each test feature and all training features
        dists = torch.cdist(features_test, self.features_train)
        
        # Obtain indices of the k smallest distances for each test feature
        _, indices = dists.topk(self.k, dim=1, largest=False)
        
        y_pred = []
        
        # For each set of k nearest neighbors, determine the most common label
        for index_set in indices:
            nearest_labels = self.y_train[index_set]
            most_common = Counter(nearest_labels.numpy()).most_common(1)
            y_pred.append(most_common[0][0])
        
        # Convert list of predicted labels to tensor and return
        return torch.tensor(y_pred)

    def _hog(self, X):
        # Internal helper method to compute HOG feature for a given image X
        image = transforms.functional.to_pil_image(X)
        fd = hog(image,
                 orientations=8,
                 pixels_per_cell=(16, 16),
                 cells_per_block=(1, 1),
                 visualize=False,
                 channel_axis=-1,
                 feature_vector=True)
        return fd
    

**FRAGE**: Was macht dieses Modell? Interpretieren Sie die Klasse `HOGClassifier`

**FRAGE**: Welche Modell-Parameter werden gelernt / optimisiert?

**FRAGE**: Für welches Modell erwarten Sie die höhere Genauigkeit?

### Evaluierung  und Modell-Selektion

In dieser Sektion geht es darum zu definieren, wie ein Modell gemessen wird (Metrik). Eine gängige Metrik für Klassifikationsprobleme ist die `Accuracy`. Diese beschreibt den Anteil der korrekten Klassifikationen und ist somit im Range $[0-1]$.

Ausserdem geht es darum mehrere Modelle zu vergleichen und das Beste zu identifizieren. 

Dazu teilen wir die Trainings-Daten in einen Train- und einen Validation-Split.

In [ ]:
from torch.utils.data import random_split

# Define the lengths
train_length = int(0.8 * len(trainset)) # 80% of the dataset for training
val_length = len(trainset) - train_length # remaining for validation

# Split the dataset
rng = torch.Generator().manual_seed(123)
train_split, validation_split = random_split(trainset, [train_length, val_length], generator=rng)

print(f"train_split size: {len(train_split)}")
print(f"validation_split size: {len(validation_split)}")

Nun laden wir die Daten in Memory.

In [ ]:
trainloader = torch.utils.data.DataLoader(train_split, batch_size=len(train_split), shuffle=False)
Xtrain, ytrain = next(iter(trainloader))

Wir definieren die Modelle.

In [ ]:
models = {
    'centroid': CentroidClassifier(num_classes=10, shape=(3, 32, 32)),
    'hog_classifier': HOGClassifier(k=20),
    'nearest_neighbour': ClosestNeighborClassifier()
}

Nun trainieren / fitten wir die Modelle:

In [ ]:
for name, model in models.items():
    print(f"Fitting: {name}")
    model = model.fit(Xtrain, ytrain)

**FRAGE:** Welches Modell hat am längest gebraucht und warum?

Nun laden wir die Validierungsdaten.

In [ ]:
valloader = torch.utils.data.DataLoader(validation_split, batch_size=1024, shuffle=False)   

Nun generieren wir Vorhersagen / Predictions für die Validierungs-Daten.

In [ ]:
predictions_all = {}
for name, model in models.items():
    print(f"Predicting: {name}")
    predictions_batches = list()
    for Xbatch, ybatch in tqdm(valloader, total=len(valloader)):
        predictions_batches.append(model.predict(Xbatch))
    predictions_all[name] = torch.concat(predictions_batches)

**FRAGE:** Welches Modell hat am längest gebraucht und warum?

Nun berechnen wir die Accuracy für die Modelle.

In [ ]:
yval_true = torch.tensor([y for x, y in validation_split])

accs = {}
for name, predictions in predictions_all.items():
    acc = (predictions == yval_true).to(torch.float).mean()
    accs[name] = acc
    print(f"Accuracy {name}: {acc:.3f}")

Will man die Performance des besten Modelles abschätzen evaluiert man es oft nochmal auf einem Testdatensatz. Dieser darf nicht in der Modell-Selektion verwendet worden sein. Dadurch erhält man eine genauere Einschätzung der Performance. 

In [ ]:
testloader = torch.utils.data.DataLoader(testset, batch_size=1024, shuffle=False)
ytest_true = torch.tensor([y for x, y in testset])

predictions_batches = list()

best_model_name = max(accs, key=accs.get)
print(f"best model: {best_model_name}")

best_model = models[best_model_name]
for Xbatch, ybatch in tqdm(testloader, total=len(testloader)):
    predictions_batches.append(best_model.predict(Xbatch))
predictions_all = torch.concat(predictions_batches)


acc = (predictions_all == ytest_true).to(torch.float).mean()
print(f"Accuracy: {acc:.3f}")

### (Optional) Weitere Aufgaben

- Erstellen Sie ein weiteres Modell und vergleichen Sie dessen Performance.

- Visualisieren Sie die `nearest neighbours` von verschiedenen Datenpunkten.

